# Decorators
## What are decorators?
"Decoration is a way to specify management code for functions and classes." ... "A decorator itself is a callable that returns a callable." - Mark Lutz

A decorator wraps a function without modifying the function itself. The result of the wrapping?
* Adds functionality to the function.
* Modifies the behavior of the function.

A simple example probably illustrates this well:

In [ ]:
def my_decorator(func):
    def new_func(n):
        return '$' + str(func(n))
    return new_func

In [ ]:
def my_function(a):
    return(a)

In [ ]:
print(my_function(100))
type(my_function(100))

In [ ]:
result = my_decorator(my_function)(100)
print(result)
type(result)

This is nice, but there is shorthand for this approach

In [ ]:
@my_decorator
def my_function(a):
    return(a)

In [ ]:
# call the decorated function
print(my_function(100))
type(my_function(100))

## Our use case: tasks that leverage Dask
In our use case (MPI/OpenMP-enabled tasks) we want to use Dask as a task manager. This requires a lot of tweaking based on the machine resources (how many OpenMP threads, how many nodes for a given `ncores`, what additional environment variables are required,...).

The necessary logic for this can all be hidden away, and decorators are a great way for us to do this while ensuring what we do is compatible with Dask.

In [ ]:
from dask.distributed import LocalCluster

In [ ]:
from jobqueue_features.decorators import on_cluster, task
from jobqueue_features.functions import set_default_cluster
from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.clusters_controller import (
    clusters_controller_singleton as controller,
)

In [ ]:
set_default_cluster(LocalCluster)

In [ ]:
@task()
def square(x):
    return x ** 2

In [ ]:
@on_cluster()
def simple_taskset():
    sq_tasks = list(map(square, range(1, 11)))
    return sq_tasks

In [ ]:
%%time
sq_tasks=simple_taskset()

Because we've used decorators this is no longer a list of integers, but a list of *futures*, which are basically promises to return a result:

In [ ]:
sq_tasks

When we actually want the results, we need to request them using the `result` method. `result` is a blocking operation, the code block won't continue until it has successfully gathered the results. 

In [ ]:
for my_task in sq_tasks:
    print(my_task.result())

Now let's clean up our cluster since we don't need it any more. We can use our special `controller` to delete all running clusters.

In [ ]:
controller._close()

## Using a custom cluster type
In the last cast case we used a `LocalCluster`, now let's repeat the exercise but with our custom cluster type

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="myCluster"
)

This cluster type leverages `dask_jobqueue` and will actually submit a job to the queueing system.

In [ ]:
print(custom_cluster.job_script())

Now let's define our task again (slightly differently but that is not important)

In [ ]:
@on_cluster(cluster=custom_cluster)
@task(cluster=custom_cluster)
def square(x):
    return x ** 2

In [ ]:
def simple_taskset():
    sq_tasks = list(map(square, range(1, 11)))
    print([t.result() for t in sq_tasks])

In [ ]:
%%time
simple_taskset()

## Exercise

1. Reuse the existing cluster and create another task that returns the hostname of node it is running on (using `os.getenv("HOSTNAME")`) and decorate it with `@on_cluster` and `@task`. Both decorators can be stacked. 
2. Run it and print the result.

In [ ]:
import os
# Here's what we get where we are now
os.getenv("HOSTNAME")

In [ ]:
# @...
# @...
# def ...

# execute...
# ... and get result

Let's clean up after ourselves again

In [ ]:
controller._close()